Packages

In [145]:
import pandas as pd
from openpyxl import load_workbook

Parameters

In [146]:
Horizon = 2040
start_year = 2024
LeapYearsInvestment = 5
total_periods = int((Horizon - start_year)/LeapYearsInvestment)

bus_map = load_workbook('data_pypsa/bus_map30.xlsx').active
new_names = [cell.value for cell in bus_map['A'][1:]]

General.xlsx

In [147]:
workbook = load_workbook('Data handler/bolivia_v1/General.xlsx')

# seasonScale


# CO2Cap
pypsa_emissions = load_workbook('data_pypsa/emissions.xlsx').active
co2_cap_sheet = workbook['CO2Cap']

emission_year = 2024
start_value = pypsa_emissions['A2'].value
end_year = 2050
end_value = 0

years = range(emission_year, end_year + 1)
emissions_trajectory = []

decrease_per_year = (start_value - end_value) / (end_year - emission_year)

for i in years:
    emission = max(start_value - decrease_per_year * (i - emission_year), end_value)
    emissions_trajectory.append(emission)

for i in range(total_periods):
    co2_cap_sheet.cell(row=i+4, column=1, value= i + 1)
    co2_cap_sheet.cell(row=i+4, column=2, value=emissions_trajectory[i * LeapYearsInvestment])

for i in range(total_periods+4, co2_cap_sheet.max_row + 1):
    for col in range(1, 3): 
        if co2_cap_sheet.cell(row=i, column=col).value is not None:
            co2_cap_sheet.cell(row=i, column=col).value = None

# CO2Price
co2_price_sheet = workbook['CO2Price']
for i in range(total_periods):
    co2_price_sheet.cell(row=i+4, column=1, value= i + 1)
    co2_price_sheet.cell(row=i+4, column=2, value=0)

for i in range(total_periods+4, co2_price_sheet.max_row + 1):
    for col in range(1, 3): 
        if co2_price_sheet.cell(row=i, column=col).value is not None:
            co2_price_sheet.cell(row=i, column=col).value = None

workbook.save('Data handler/bolivia_v1/General.xlsx')


Generator.xlsx

In [148]:
pypsa_tech = load_workbook('data_pypsa/technologies.xlsx').active

Node.xlsx

In [149]:
workbook = load_workbook('Data handler/bolivia_v1/Node.xlsx')
pypsa_load = load_workbook('data_pypsa/load_per_gen30.xlsx').active
load_dict = {row[0]: row[1] for row in pypsa_load.iter_rows(min_row=2, values_only=True)}

for sheet in workbook.sheetnames:
    current_sheet = workbook[sheet]

    nodes_positions = {'A3': ('B3', 1), 'E3': ('F3', 5)}
    
    for node_cell_ref, (period_cell_ref, col_index) in nodes_positions.items():
        if current_sheet[node_cell_ref].value == 'Nodes':
            # Check if the cell next to "Nodes" contains "Period"
            if current_sheet[period_cell_ref].value == 'Period':
                # Logic for handling periods
                start_row = current_sheet[node_cell_ref].row + 1
                
                # Repeat names for each period
                for period_index in range(total_periods):
                    period_start_row = start_row + (period_index * len(new_names))
                    
                    # Fill names for the current period
                    for i, name in enumerate(new_names, start=period_start_row):
                        current_sheet.cell(row=i, column=col_index, value=name)
                        current_sheet.cell(row=i, column=col_index + 1, value=period_index + 1)

                        if period_index == 0:
                            current_sheet.cell(row=i, column=col_index + 2, value=load_dict[name])
                            

                # Clear cells after the last period's names, if necessary
                last_name_row = start_row + (total_periods * len(new_names))
                for i in range(last_name_row, current_sheet.max_row + 1):
                    if current_sheet.cell(row=i, column=col_index).value or current_sheet.cell(row=i, column=col_index + 1).value:
                        current_sheet.cell(row=i, column=col_index).value = None
                        current_sheet.cell(row=i, column=col_index + 1).value = None
            else:
                max_row = current_sheet[node_cell_ref].row + len(new_names)
                # Fill names directly if no "Period" is next to "Nodes"
                for i, name in enumerate(new_names, start=current_sheet[node_cell_ref].row + 1):
                    current_sheet.cell(row=i, column=col_index, value=name)

                for i in range(max_row + 1, current_sheet.max_row + 1):
                    if current_sheet.cell(row=i, column=col_index).value is not None:
                        current_sheet.cell(row=i, column=col_index).value = None

workbook.save('Data handler/bolivia_v1/Node.xlsx')



Sets.xlsx

In [150]:
file_path = 'Data handler/bolivia_v1/Sets.xlsx'
workbook = load_workbook(file_path)

# Nodes
node_sheet = workbook['Nodes']

max_row = node_sheet['A1'].row + len(new_names)
# Fill names directly if no "Period" is next to "Nodes"
for i, name in enumerate(new_names, start=node_sheet['A1'].row + 1):
    node_sheet.cell(row=i, column=1, value=name)

for i in range(max_row + 1, node_sheet.max_row + 1):
    if node_sheet.cell(row=i, column=1).value is not None:
        node_sheet.cell(row=i, column=1).value = None


# OffshoreNodes
offshore_node_sheet = workbook['OffshoreNodes']
max_row = offshore_node_sheet.max_row

for i in range(1, max_row):
    if offshore_node_sheet.cell(row=i + 1, column=1).value is not None:
        offshore_node_sheet.cell(row=i + 1, column=1).value = None


# Horizon


# Storage
storage_sheet = workbook['Storage']
storage_sheet.cell(row=2, column=1, value='Li-Ion_BESS')
storage_sheet.cell(row=3, column=1).value=None

# Technology
techs = ['Existing', 'Gas', 'Oil', 'Bio', 'Geo', 'Hydro_reg', 'Hydro_ror', 'Wind_onshr', 'Solar']
tech_sheet = workbook['Technology']
for i in range(len(techs)):
    tech_sheet.cell(row=i + 2, column=1, value=techs[i])

# Generators
generators_sheet = workbook['Generators']
for index, row in enumerate(pypsa_tech.iter_rows(min_row=2, values_only=True), start=1):
    generators_sheet.cell(row=index + 1, column=1, value=row[2])

max_row = 1 + len(pypsa_tech['C'])
for i in range(max_row, generators_sheet.max_row + 1):
    if generators_sheet.cell(row=i, column=1).value is not None:
        generators_sheet.cell(row=i, column=1).value = None

thermal_gens = ['Gas OCGT', 'Bio', 'Gas CCGT', 'Oil existing', 'Geo']
for i in range(len(thermal_gens)):
    generators_sheet.cell(row=i + 2, column=4, value=thermal_gens[i])

max_row = 1 + len(thermal_gens)
for i in range(max_row + 1, generators_sheet.max_row + 1):
    if generators_sheet.cell(row=i, column=4).value is not None:
        generators_sheet.cell(row=i, column=4).value = None

# LineType
line_sheet = workbook['LineType']
line_sheet.cell(row=2, column=1, value='HVAC_OverheadLine')
line_sheet.cell(row=3, column=1).value=None

# HourOfSeason


# StorageOfNodes
storage_node_sheet = workbook['StorageOfNodes']
max_row = storage_node_sheet['A3'].row + len(new_names)

for i, name in enumerate(new_names, start=storage_node_sheet['A3'].row + 1):
    storage_node_sheet.cell(row=i, column=1, value=name)
    storage_node_sheet.cell(row=i, column=2, value=storage_sheet['A2'].value)

for i in range(max_row + 1, storage_node_sheet.max_row + 1):
    for col in range(1, 3): 
        if storage_node_sheet.cell(row=i, column=col).value is not None:
            storage_node_sheet.cell(row=i, column=col).value = None

# DirectionalLines
line_connections = workbook['DirectionalLines']
pypsa_connections = load_workbook('data_pypsa/lines30.xlsx').active

for index, row in enumerate(pypsa_connections.iter_rows(min_row=2, values_only=True), start=1):
    value_a, value_b = row[1], row[2]

    line_connections.cell(row=index + 3, column=1, value=value_a)
    line_connections.cell(row=index + 3, column=2, value=value_b)

max_row = 2 + len(pypsa_connections['A'])
for i in range(max_row + 1, line_connections.max_row + 1):
    for col in range(1, 4): 
        if line_connections.cell(row=i, column=col).value is not None:
            line_connections.cell(row=i, column=col).value = None

# LineTypeOfDirectionalLines
line_type_connections = workbook['LineTypeOfDirectionalLines']

for index, row in enumerate(pypsa_connections.iter_rows(min_row=2, values_only=True), start=1):
    value_a, value_b = row[1], row[2]

    line_type_connections.cell(row=index + 3, column=1, value=value_a)
    line_type_connections.cell(row=index + 3, column=2, value=value_b)
    line_type_connections.cell(row=index + 3, column=3, value='HVAC_OverheadLine')

for i in range(max_row + 1, line_type_connections.max_row + 1):
    for col in range(1, 4): 
        if line_type_connections.cell(row=i, column=col).value is not None:
            line_type_connections.cell(row=i, column=col).value = None


# GeneratorsOfNode
generators_node_sheet = workbook['GeneratorsOfNode']
pypsa_generators = load_workbook('data_pypsa/generators30.xlsx').active
for g_index, g_row in enumerate(pypsa_generators.iter_rows(min_row=2, max_row=84 ,min_col=9, max_col=10, values_only=True), start=1):
    generators_node_sheet.cell(row=g_index +3, column=1, value=g_row[1])
    for t_index, tech in enumerate(pypsa_tech.iter_rows(min_row=2, min_col=3, max_col=4,values_only=True), start=1):
        if g_row[0] == tech[1]:
            generators_node_sheet.cell(row=g_index +3, column=2).value = tech[0]
            break
            

max_row = 85
for i in range(max_row + 1, generators_node_sheet.max_row + 1):
    for col in range(1, 3): 
        if generators_node_sheet.cell(row=i, column=col).value is not None:
            generators_node_sheet.cell(row=i, column=col).value = None
    
    

# GeneratorsOfTechnology
generator_tech_sheet = workbook['GeneratorsOfTechnology']
generator_tech_dict = {'Gas OCGT': ['Gas'], 'Gas CCGT': ['Gas'], 'Oil existing': ['Existing', 'Oil'], 'Hydro regulated': ['Hydro_reg'], 'Hydro run-of-river': ['Hydro_ror'], 'Bio': ['Bio'], 'Geo': ['Geo'], 'Wind onshore': ['Wind_onshr'], 'Solar': ['Solar']}
row_num = 4
for key, items in generator_tech_dict.items():
    for item in items:
        generator_tech_sheet.cell(row=row_num, column=2, value=key)
        generator_tech_sheet.cell(row=row_num, column=1, value=item)
        row_num += 1

for i in range(len(generator_tech_dict) + 4, generator_tech_sheet.max_row + 1):
    for col in range(1, 3): 
        if generator_tech_sheet.cell(row=i, column=col).value is not None:
            generator_tech_sheet.cell(row=i, column=col).value = None
    


# Coords
coord_sheet = workbook['Coords']
pypsa_coords = load_workbook('data_pypsa/buses_EMPIRE.xlsx').active

for index, row in enumerate(pypsa_coords.iter_rows(min_row=2, values_only=True), start=1):
    node, x, y = row[0], row[3], row[4]
    coord_sheet.cell(row=index + 3, column=1, value=node)
    coord_sheet.cell(row=index + 3, column=2, value=y)
    coord_sheet.cell(row=index + 3, column=3, value=x)

for i in range(4 + len(new_names), coord_sheet.max_row + 1):
    for col in range(1, 4): 
        if coord_sheet.cell(row=i, column=col).value is not None:
            coord_sheet.cell(row=i, column=col).value = None

workbook.save(file_path)